<a href="https://colab.research.google.com/github/Jagoda222/LoLa---group-8/blob/main/model_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install accelerate -U
!pip install datasets==2.14.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver doe

In [12]:
import torch
from os import path as op
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import datasets
import random
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [24]:
import pandas as pd

# Load the dataset (assuming it's in a CSV file)
# Replace "snli_train.csv" with the path to your actual dataset
snli_data = load_dataset("snli")
data = pd.DataFrame(snli_data['train'])

# Step 1: Group data into triplets (each triplet is 3 rows)
# We'll assume that every 3 consecutive rows form a triplet
data['triplet_id'] = data.index // 3  # Assign a triplet ID to every group of 3 rows

# Step 2: Randomly sample 666 triplets
triplet_ids = data['triplet_id'].unique()  # Get unique triplet IDs
sampled_triplet_ids = pd.Series(triplet_ids).sample(n=666, random_state=42)  # Randomly select 666 triplet IDs

# Step 3: Filter the dataset to keep only the sampled triplets
reduced_data = data[data['triplet_id'].isin(sampled_triplet_ids)]

# Drop the helper column (triplet_id) before saving
reduced_data = reduced_data.drop(columns=['triplet_id'])

# Step 4: Save the reduced dataset

print(reduced_data.head())


                                               premise  \
162                   Children going home from school.   
163                   Children going home from school.   
164                   Children going home from school.   
1719  A man is putting up a poster in front of a shop.   
1720  A man is putting up a poster in front of a shop.   

                                       hypothesis  label  
162              The children are at the library.      2  
163                The school children head home.      0  
164   The children are walking in the afternooon.      1  
1719     A man hangs a poster in front of a shop.      0  
1720         There is a man in front of the shop.      0  


In [28]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, f1_score



# Tokenize datasets
model_name = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding=True)

# Tokenize reduced training data
train_dataset = Dataset.from_pandas(reduced_data)
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)

# Tokenize validation and test sets
val_dataset = snli_data["validation"].map(tokenize_function, batched=True)
test_dataset = snli_data["test"].map(tokenize_function, batched=True)

# Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Hyperparameter search space
learning_rates = [1e-5, 2e-5, 3e-5]
batch_sizes = [16, 32]
weight_decays = [0.0, 0.01]
gradient_accumulation_steps = [1, 2]

results = []

# Loop through hyperparameter combinations
for lr in learning_rates:
    for batch_size in batch_sizes:
        for wd in weight_decays:
            for grad_accum in gradient_accumulation_steps:
                print(f"Training with lr={lr}, batch_size={batch_size}, weight_decay={wd}, grad_accum_steps={grad_accum}")

                # Training arguments
                training_args = TrainingArguments(
                    output_dir="./results",  # Directory to save the model
                    evaluation_strategy="steps",  # Evaluate every eval_steps
                    save_strategy="steps",  # Save checkpoints at the same interval as evaluation
                    eval_steps=500,  # Fixed as per requirements
                    save_steps=500,  # Save the model every 500 steps
                    logging_steps=500,  # Log every 500 steps
                    num_train_epochs=1,  # Fixed to 1 epoch
                    per_device_train_batch_size=batch_size,
                    per_device_eval_batch_size=batch_size,
                    learning_rate=lr,
                    weight_decay=wd,
                    gradient_accumulation_steps=grad_accum,
                    load_best_model_at_end=True,  # Load the best model
                )

                # Initialize the model
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

                # Initialize Trainer
                trainer = Trainer(
                    model=model,
                    args=training_args,
                    train_dataset=tokenized_train_dataset,
                    eval_dataset=val_dataset,
                    tokenizer=tokenizer,
                    compute_metrics=compute_metrics,
                )

                # Train the model
                trainer.train()

                # Evaluate on validation set
                val_metrics = trainer.evaluate(eval_dataset=val_dataset)
                val_metrics["learning_rate"] = lr
                val_metrics["batch_size"] = batch_size
                val_metrics["weight_decay"] = wd
                val_metrics["gradient_accumulation_steps"] = grad_accum
                val_metrics["set"] = "validation"

                # Evaluate on test set
                test_metrics = trainer.evaluate(eval_dataset=test_dataset)
                test_metrics["learning_rate"] = lr
                test_metrics["batch_size"] = batch_size
                test_metrics["weight_decay"] = wd
                test_metrics["gradient_accumulation_steps"] = grad_accum
                test_metrics["set"] = "test"

                # Save metrics
                results.append(val_metrics)
                results.append(test_metrics)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display results




/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/1998 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Training with lr=1e-05, batch_size=16, weight_decay=0.0, grad_accum_steps=1


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-28-1bb0f24a1304>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-28-1bb0f24a1304>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


    eval_loss  eval_accuracy   eval_f1  eval_runtime  eval_samples_per_second  \
0    1.095578         0.3544  0.209852       29.9110                  334.326   
1    1.095491         0.3560  0.208789       26.6925                  374.637   
2    1.096384         0.3329  0.166288       29.8038                  335.528   
3    1.096280         0.3368  0.169710       26.4751                  377.713   
4    1.092697         0.5408  0.463806       32.0232                  312.274   
5    1.092737         0.5380  0.462383       27.4580                  364.193   
6    1.096384         0.3329  0.166288       32.0950                  311.575   
7    1.096281         0.3368  0.169710       27.8677                  358.838   
8    1.096125         0.3329  0.166288       30.4048                  328.895   
9    1.095989         0.3368  0.169710       26.0442                  383.962   
10   1.097316         0.3329  0.166288       30.0279                  333.024   
11   1.097115         0.3368

ValueError: Could not interpret value `accuracy` for `y`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>

In [33]:
print(results_df)


    eval_loss  eval_accuracy   eval_f1  eval_runtime  eval_samples_per_second  \
0    1.095578         0.3544  0.209852       29.9110                  334.326   
1    1.095491         0.3560  0.208789       26.6925                  374.637   
2    1.096384         0.3329  0.166288       29.8038                  335.528   
3    1.096280         0.3368  0.169710       26.4751                  377.713   
4    1.092697         0.5408  0.463806       32.0232                  312.274   
5    1.092737         0.5380  0.462383       27.4580                  364.193   
6    1.096384         0.3329  0.166288       32.0950                  311.575   
7    1.096281         0.3368  0.169710       27.8677                  358.838   
8    1.096125         0.3329  0.166288       30.4048                  328.895   
9    1.095989         0.3368  0.169710       26.0442                  383.962   
10   1.097316         0.3329  0.166288       30.0279                  333.024   
11   1.097115         0.3368